In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load the data
df = pd.read_csv('data/census.csv', sep=',\s', engine='python')

# Display the first few rows of the DataFrame to verify that it loaded correctly
df.head()


In [ ]:
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

In [ ]:
# Drop rows with missing values
df.dropna(inplace=True)

In [ ]:
# Strip leading and trailing spaces from string columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('data/cleaned_census_income.csv', index=False)

In [3]:
%pip install ml

ERROR: Could not find a version that satisfies the requirement ml (from versions: none)
ERROR: No matching distribution found for ml
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import train_test_split
from ml import clean_data
from ml.data import process_data
from ml.model import train_model, inference,compute_model_metrics,process_slices
import logging
import joblib

In [3]:
# log config 
logging.basicConfig(filename='logs/log',level=logging.INFO,filemode='w')

# Add code to load in the data.
data = clean_data.cleaned_data()

# Optional enhancement, use K-fold cross validation instead of a train-test split.
train, test = train_test_split(data, test_size=0.20)
logging.info("Train Test data split done")

cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

X_train, y_train, encoder, lb = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)

# Train and save a model.
model = train_model(X_train,y_train)
joblib.dump(model,'model/trained_model.joblib')



/home/t0142f5/miniconda3/envs/udacity_ml_ops_3/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


['model/trained_model.joblib']

In [4]:
# Proces the test data with the process_data function.
X_test, y_test, encoder_test, lb_test = process_data(
    test, categorical_features=cat_features, label="salary", training=False, encoder=encoder, lb=lb
)

joblib.dump(encoder_test,'model/encoder.joblib')
joblib.dump(lb_test,'model/lb.joblib')

pred = inference(model,X_test)

precision, recall, fbeta = compute_model_metrics(y_test,pred)

logging.info(f"Precision: {precision}, Recall: {recall}, Fbeta: {fbeta}")


In [5]:
# performance of model on slices of dat
slice_metrics = process_slices(test, model, cat_features, 'education', encoder, lb)

logging.info(f"slice_metrics: {slice_metrics}")